In [4]:
import pandas as pd

# Load data from CSV file
df = pd.read_csv(r'2023_20mins.csv')

# Filter out entries with Frequency less than 100
df = df[df['Frequency'] >= 600]

# Remove entries where the start and end stations are the same
df = df[df['Station1'] != df['Station2']]

# Save the cleaned data to a new CSV file
df.to_csv('20_data.csv', index=False)

print("Data cleaning completed. The cleaned data has been saved to 'cleaned_data.csv'.")


Data cleaning completed. The cleaned data has been saved to 'cleaned_data.csv'.


In [5]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

# 讀取台北市行政區界線資料（假設為'taipei_districts.geojson'）
districts = gpd.read_file('taipei.geojson')

# 讀取你的資料
data = pd.read_csv('2023_AllData.csv')

# 創建地理資料框
geometry1 = [Point(xy) for xy in zip(data['Station1_lon'], data['Station1_lat'])]
geometry2 = [Point(xy) for xy in zip(data['Station2_lon'], data['Station2_lat'])]
gdf1 = gpd.GeoDataFrame(data, geometry=geometry1)
gdf2 = gpd.GeoDataFrame(data, geometry=geometry2)

# 設定坐標參考系統（假設為WGS 84）
gdf1.crs = {'init': 'epsg:4326'}
gdf2.crs = {'init': 'epsg:4326'}
districts.crs = {'init': 'epsg:4326'}

# 進行空間匹配，找出每個站點所在的行政區
gdf1 = gpd.sjoin(gdf1, districts, how="left", op='intersects')
gdf2 = gpd.sjoin(gdf2, districts, how="left", op='intersects')

# 提取行政區名稱
gdf1['Station1_district'] = gdf1['TOWNNAME']
gdf2['Station2_district'] = gdf2['TOWNNAME']

# 合併資料
data['Station1_district'] = gdf1['Station1_district']
data['Station2_district'] = gdf2['Station2_district']

# 創建行政區矩陣
districts_list = districts['TOWNNAME'].unique()
matrix = pd.DataFrame(0, index=districts_list, columns=districts_list)

# 計算移動次數
for i in range(len(data)):
    start_district = data.loc[i, 'Station1_district']
    end_district = data.loc[i, 'Station2_district']
    frequency = data.loc[i, 'Frequency']
    if pd.notna(start_district) and pd.notna(end_district):
        matrix.loc[start_district, end_district] += frequency

# 顯示結果
print(matrix)


c:\Users\chuan\miniconda3\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\chuan\miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3377: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):
c:\Users\chuan\miniconda3\lib\site-packages\IPython\core\interactiveshell.py:3377: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


         大安區     文山區      信義區     萬華區      中正區     南港區      松山區     大同區  \
大安區  5867066   62082   362202   13537   456775     334   165416    5303   
文山區   135912  638738      792     260    39767       0        3       0   
信義區   500636    2074  2600446       1     2605   92223   143377       9   
萬華區    10571     179        0  747563   167660       0        1   12589   
中正區   655489   17569     1448  350546  1400212       0     9907   88048   
南港區      608       0   187507       0        1  824391    73968       0   
松山區   200080      16   305341      15     9034   24838  1395189    2764   
大同區     3008       1        2   59887    45561       1     3007  899087   
中山區   134542      12     2572    5642   122768       6   290507  276354   
內湖區       54       0     6689       0        0   16408    21848       1   
士林區        1       0        1      16       77       3       21   39400   
北投區        0       0        1       0        0       0        1      28   

         中山區      內湖區   

In [7]:
matrix_sum = matrix.sum().sum()
print("矩陣總和: ", matrix_sum)
matrix.to_csv('matrix.csv')

矩陣總和:  27041297
